In [1]:
FOLDER = '../data/stl'
EXTENSION = '.stl'
EXAMPLE_FILE = '../data/stl/020_l.stl'
EXAMPLE_EXTRA_LMRK = '../data/stl/020_l-extra.pkl'
OUTPUT_FOLDER = 'output/extra-lmrk'

In [2]:
import sys
sys.path.append('..')

## Load data

In [3]:
import os

files = os.listdir(FOLDER)
files = [os.path.join(FOLDER, f) for f in files if EXTENSION in f]
files.sort()
files

['../data/stl/020_l.stl',
 '../data/stl/15032021_001_l 1.stl',
 '../data/stl/15032021_001_r 1.stl']

In [4]:
# load landmarks labelling
import pandas as pd

lmrk_dict = {}

for file in files:
    lmrk = pd.read_pickle(file.replace(EXTENSION, '.pkl'))
    lmrk_dict[file] = lmrk

lmrk_dict

{'../data/stl/020_l.stl': coord              x           y          z
 landmark                                   
 P1         69.653463   54.673492  -7.735388
 P10      -168.788818   79.678271 -11.032859
 P11      -122.558701  102.675117 -60.435042
 P12      -123.716329   31.840291 -67.462064
 P2         59.080175   94.482407  -9.113344
 P3         18.191420    5.494654 -10.807760
 P4         19.126142   99.146723 -18.009934
 P5        -15.352643    2.470859 -11.472470
 P6        -34.886927   64.674735 -62.592272
 P7        -53.684757   68.070063 -71.413231
 P8       -130.517046  102.431932  -6.028035
 P9       -144.132384   39.577845  -8.140953,
 '../data/stl/15032021_001_l 1.stl': coord              x           y          z
 landmark                                   
 P1         55.771399   71.333448  -7.274359
 P10      -180.955130   63.650910 -12.920648
 P11      -124.075143   97.695019 -63.217842
 P12      -121.894275   28.415395 -55.086196
 P2         27.630023  112.138113  -7.

In [5]:
# load meshes
import pyvista as pv

mesh_dict = {}

for file in files:
    mesh = pv.read(file)
    mesh_dict[file] = mesh

mesh_dict

{'../data/stl/020_l.stl': PolyData (0x7fd7e98b7fa0)
   N Cells:    548852
   N Points:   274424
   N Strips:   0
   X Bounds:   -1.709e+02, 8.033e+01
   Y Bounds:   1.213e+00, 1.057e+02
   Z Bounds:   -1.241e+02, 8.183e-01
   N Arrays:   0,
 '../data/stl/15032021_001_l 1.stl': PolyData (0x7fd7e98c6040)
   N Cells:    519070
   N Points:   259537
   N Strips:   0
   X Bounds:   -1.840e+02, 6.258e+01
   Y Bounds:   9.439e+00, 1.149e+02
   Z Bounds:   -1.171e+02, 1.312e+00
   N Arrays:   0,
 '../data/stl/15032021_001_r 1.stl': PolyData (0x7fd7e98c60a0)
   N Cells:    499534
   N Points:   249767
   N Strips:   0
   X Bounds:   -6.845e+01, 1.812e+02
   Y Bounds:   -1.017e+02, 4.561e+00
   Z Bounds:   -1.170e+02, 8.008e-01
   N Arrays:   0}

In [6]:
# load example mesh and the extra landmarks labelling
example_mesh = mesh_dict[EXAMPLE_FILE]
example_extra_lmrk = pd.read_pickle(EXAMPLE_EXTRA_LMRK)
example_extra_lmrk

coord,x,y,z
landmark,,,
AA,-69.571717,97.848721,-14.324256
LMB,-56.088392,18.053136,-0.889815
MLA-AP,-6.568192,87.054788,-0.614215
MLA-PP,-109.917336,94.292150,-0.881860
MMB,-49.540290,60.708933,-1.221400


## Landmarks motion interpolation

In [7]:
lmrk_dict['../data/stl/15032021_001_l 1.stl'].to_numpy().shape

(12, 3)

In [8]:
import numpy as np
from mesh4d.analyse import measure
from scipy.interpolate import RBFInterpolator

def udmc(source_lmrk: pd.DataFrame, target_lmrk: pd.DataFrame, source_mesh: pv.core.pointset.PolyData, target_mesh: pv.core.pointset.PolyData, post_align: bool = True, post_nbr: int = 100) -> RBFInterpolator:
    # init motion field
    field = RBFInterpolator(
        source_lmrk.to_numpy(),  # (N, 3)
        target_lmrk.to_numpy(),  # (N, 3)
    )

    # post alignment
    if post_align:
        source_points = np.array(source_mesh.points)  # (N_all, 3)
        shift_points = field(source_points)
        deform_points = measure.nearest_points_from_plane(target_mesh, shift_points)
        field = RBFInterpolator(source_points, deform_points, neighbors=post_nbr)

    return field

In [9]:
from tqdm import tqdm

field_dict = {}
extra_lmrk_dict = {}

for file in tqdm(files):
    field = udmc(
        source_lmrk=lmrk_dict[EXAMPLE_FILE],
        target_lmrk=lmrk_dict[file],
        source_mesh=mesh_dict[EXAMPLE_FILE],
        target_mesh=mesh_dict[file],
        # post_align=False,
        post_align=True,
        )
    field_dict[file] = field
    extra_lmrk = pd.DataFrame(
        field(example_extra_lmrk.to_numpy()),
        columns=example_extra_lmrk.columns,
        index=example_extra_lmrk.index,
        )
    extra_lmrk_dict[file] = extra_lmrk
    extra_lmrk.to_pickle(file.replace(EXTENSION, '-extra-udmc.pkl'))

extra_lmrk_dict

100%|██████████| 3/3 [00:34<00:00, 11.62s/it]


{'../data/stl/020_l.stl': coord              x          y          z
 landmark                                  
 AA        -69.571717  97.848721 -14.324256
 LMB       -56.088392  18.053136  -0.889815
 MLA-AP     -6.568192  87.054788  -0.614215
 MLA-PP   -109.917336  94.292150  -0.881860
 MMB       -49.540290  60.708933  -1.221400,
 '../data/stl/15032021_001_l 1.stl': coord              x           y          z
 landmark                                   
 AA        -87.358214  103.641582 -19.437133
 LMB       -63.135306   19.890624  -0.140206
 MLA-AP    -26.829123   97.612372  -0.474042
 MLA-PP   -130.350246   87.151040  -2.262987
 MMB       -64.742536   62.091391  -0.058050,
 '../data/stl/15032021_001_r 1.stl': coord             x          y          z
 landmark                                 
 AA        43.202649 -99.796393 -12.789841
 LMB       47.901470  -9.914901  -0.337727
 MLA-AP    97.693706 -80.236501  -0.518221
 MLA-PP    -0.968466 -83.670049   0.197210
 MMB       55.595937

## Visual checks

In [10]:
scene = pv.Plotter()
scene.add_mesh(example_mesh, opacity=1)

scene.add_points(
    lmrk_dict[EXAMPLE_FILE].to_numpy(),
    render_points_as_spheres=True,
    style='points', color='teal', point_size=20, opacity=0.9,
    )

scene.add_points(
    example_extra_lmrk.to_numpy(),
    render_points_as_spheres=True,
    style='points', color='lightcoral', point_size=20, opacity=0.9,
    )

scene.camera.roll += 160
scene.show()

Widget(value="<iframe src='http://localhost:58469/index.html?ui=P_0x7fd7eaf5be20_0&reconnect=auto' style='widt…

In [11]:
os.path.join(
    OUTPUT_FOLDER,
    os.path.basename(file).replace(EXTENSION, '.png'),
    )

'output/extra-lmrk/15032021_001_r 1.png'

In [12]:
for file in files:
    scene = pv.Plotter()
    scene.add_mesh(mesh_dict[file], opacity=1)

    scene.add_points(
        lmrk_dict[file].to_numpy(),
        render_points_as_spheres=True,
        style='points', color='teal', point_size=20, opacity=0.9,
        )

    scene.add_points(
        extra_lmrk_dict[file].to_numpy(),
        render_points_as_spheres=True,
        style='points', color='lightcoral', point_size=20, opacity=0.9,
        )

    scene.camera.roll += 160
    scene.screenshot(os.path.join(
        OUTPUT_FOLDER,
        os.path.basename(file).replace(EXTENSION, '.png'),
        ))
    scene.show()

Widget(value="<iframe src='http://localhost:58469/index.html?ui=P_0x7fd7aa3233d0_1&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:58469/index.html?ui=P_0x7fd7aa34dfd0_2&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:58469/index.html?ui=P_0x7fd7eaf5b4c0_3&reconnect=auto' style='widt…